## automatic emittance meas

In [1]:
# optionally add scripts location to path
if True:
    import sys
    sys.path.append("../../../")
    sys.path.append("../../")

run_dir = "/home/physics/ml_tuning/20230821_LCLS_Injector/"
print(sys.path)


['/home/physics/rroussel/SLAC_Xopt/lcls/emittance/emittance_optimization', '/usr/local/lcls/tools/python/toolbox', '/usr/local/lcls/package/anaconda/envs/python3.9envs/rhel7/nightly/08-21-2023/lib/python39.zip', '/usr/local/lcls/package/anaconda/envs/python3.9envs/rhel7/nightly/08-21-2023/lib/python3.9', '/usr/local/lcls/package/anaconda/envs/python3.9envs/rhel7/nightly/08-21-2023/lib/python3.9/lib-dynload', '', '/usr/local/lcls/package/anaconda/envs/python3.9envs/rhel7/nightly/08-21-2023/lib/python3.9/site-packages', '../../../', '../../']


In [2]:
from scripts.image import ImageDiagnostic
import yaml

fname = "../../OTR3_config.yml"
image_diagnostic = ImageDiagnostic.parse_obj(yaml.safe_load(open(fname)))
image_diagnostic.save_image_location = run_dir
image_diagnostic.n_fitting_restarts = 2
image_diagnostic.visualize = False
image_diagnostic.background_file = run_dir + "OTRS_IN20_621_background.npy"
print(image_diagnostic.yaml())

{screen_name: 'OTRS:IN20:621', array_data_suffix: IMAGE, array_n_cols_suffix: ROI_XNP,
  array_n_rows_suffix: ROI_YNP, resolution_suffix: RESOLUTION, beam_shutter_pv: 'IOC:BSY0:MP01:MSHUTCTL',
  background_file: /home/physics/ml_tuning/20230821_LCLS_Injector/OTRS_IN20_621_background.npy,
  save_image_location: /home/physics/ml_tuning/20230821_LCLS_Injector/, roi: null,
  min_log_intensity: 4.0, bounding_box_half_width: 3.0, wait_time: 0.1, n_fitting_restarts: 2,
  visualize: false, testing: false}



In [3]:
from scripts.automatic_emittance import ScreenEmittanceMeasurement, BeamlineConfig

drift_length = 4.17
beamline_config = BeamlineConfig(
    scan_quad_pv="QUAD:IN20:525:BCTRL",
    scan_quad_range=[-7.0, 7.0],
    scan_quad_length=0.1068,
    transport_matrix_x =[[1.0,    drift_length],
        [ 0.0,    1.0]],
    transport_matrix_y = [[1.0,    drift_length],
        [ 0.0,    1.0]],
    beam_energy=0.135,
)


measurement = ScreenEmittanceMeasurement(
    image_diagnostic=image_diagnostic,
    beamline_config=beamline_config,
    run_dir=run_dir,
    minimum_log_intensity=5.0,
    wait_time=5.0,
    n_shots=5,
    n_iterations=10,
    visualize=True
)

measurement.dump_yaml()

In [4]:
from epics import caput
from time import sleep
def eval_emittance(inputs):
    global measurement
    # set PVs
    for k, v in inputs.items():
        print(f'CAPUT {k} {v}')
        caput(k, v)

    sleep(5.0)
    print("starting emittance measurement")
    results, emit_x = measurement.run()

    results["geo_mean_emit"] = np.sqrt(results["x_emittance_median"]*results["y_emittance_median"])

    return results
    

In [5]:
import pandas as pd
filename = "../../variables.csv"
VARIABLE_RANGES = pd.read_csv(filename, index_col=0, header=None).T.to_dict(orient='list')

from xopt import VOCS
VARIABLES = ["SOLN:IN20:121:BCTRL", "QUAD:IN20:121:BCTRL","QUAD:IN20:122:BCTRL"]
vocs = VOCS(
        variables = {ele: VARIABLE_RANGES[ele] for ele in VARIABLES},
        objectives = {"geo_mean_emit": "MINIMIZE"},
    )
print(vocs)

variables={'SOLN:IN20:121:BCTRL': [0.377, 0.498], 'QUAD:IN20:121:BCTRL': [-0.021, 0.021], 'QUAD:IN20:122:BCTRL': [-0.021, 0.021]} constraints={} objectives={'geo_mean_emit': 'MINIMIZE'} constants={} observables=[]


In [6]:
from xopt import Xopt, VOCS
from xopt.evaluator import Evaluator
from xopt.numerical_optimizer import LBFGSOptimizer
from xopt.generators import UpperConfidenceBoundGenerator
from xopt.generators.bayesian.models.standard import StandardModelConstructor

# remember to set use low noise prior to false!!!
model_constructor = StandardModelConstructor(use_low_noise_prior=False)
generator = UpperConfidenceBoundGenerator(
    vocs=vocs,
    model_constructor=model_constructor,
    turbo_controller="optimize"
)
generator.numerical_optimizer.max_iter = 200
# generator.max_travel_distances = [0.1] * len(vocs.variable_names)
evaluator = Evaluator(function=eval_emittance)
X = Xopt(generator=generator, evaluator=evaluator, vocs=vocs)
X.options.dump_file = run_dir + "emittance_optimization.yml"
X


            Xopt
________________________________
Version: 2.0a1+74.g9067c79
Data size: 0
Config as YAML:
xopt: {asynch: false, strict: true, dump_file: /home/physics/ml_tuning/20230821_LCLS_Injector/emittance_optimization.yml,
  max_evaluations: null}
generator:
  name: upper_confidence_bound
  model: null
  turbo_controller:
    dim: 3
    batch_size: 1
    length: 0.25
    length_min: 0.0078125
    length_max: 2.0
    failure_counter: 0
    failure_tolerance: 2
    success_counter: 0
    success_tolerance: 2
    center_x: null
    scale_factor: 2.0
    tkwargs: {dtype: torch.float64}
    minimize: true
    best_value: null
  use_cuda: false
  model_constructor:
    name: standard
    use_low_noise_prior: false
    covar_modules: {}
    mean_modules: {}
    trainable_mean_keys: []
  numerical_optimizer: {name: LBFGS, n_raw_samples: 20, n_restarts: 20, max_iter: 200}
  max_travel_distances: null
  fixed_features: null
  computation_time: {}
  n_candidates: 1
  n_monte_carlo_samples: 

In [8]:
default = {'SOLN:IN20:121:BCTRL': 0.474877290758955,
 'QUAD:IN20:121:BCTRL': -0.0048398437,
 'QUAD:IN20:122:BCTRL': 0.0018,
}

X.evaluate_data(pd.DataFrame(default, index=[0]))

CAPUT SOLN:IN20:121:BCTRL 0.474877290758955
CAPUT QUAD:IN20:121:BCTRL -0.0048398437
CAPUT QUAD:IN20:122:BCTRL 0.0018
starting emittance measurement
CAPUT QUAD:IN20:525:BCTRL -1.2461705135320407
CAPUT QUAD:IN20:525:BCTRL -4.2203540957596335


XoptError: Xopt evaluator caught exception(s):

Evaluation index 0:
Traceback (most recent call last):
  File "/usr/local/lcls/package/anaconda/envs/python3.9envs/rhel7/nightly/08-21-2023/lib/python3.9/site-packages/xopt/utils.py", line 137, in safe_call
    result = func(*args, **kwargs)
  File "/home/physics/rroussel/SLAC_Xopt/lcls/emittance/emittance_optimization/../../../scripts/automatic_emittance.py", line 63, in eval_beamsize
    sleep(self.wait_time)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lcls/package/anaconda/envs/python3.9envs/rhel7/nightly/08-21-2023/lib/python3.9/site-packages/xopt/utils.py", line 137, in safe_call
    result = func(*args, **kwargs)
  File "/tmp/ipykernel_1297464/2187975396.py", line 12, in eval_emittance
    results, emit_x = measurement.run()
  File "/home/physics/rroussel/SLAC_Xopt/lcls/emittance/emittance_optimization/../../../scripts/automatic_emittan

In [ ]:
for i in range(10):
    print(i)
    X.step()